# Introducción práctica a pandas — ETL con pandas

Este cuaderno enseña las funcionalidades más importantes de **pandas** y muestra un ejemplo práctico de **ETL** (Extract, Transform, Load)
usando un archivo CSV de ejemplo. Al final exportamos la tabla lista para subir a una base de datos (SQLite en el ejemplo).

## Objetivos

- Presentar conceptos clave: `Series`, `DataFrame`, lectura/escritura, inspección y limpieza.
- Transformaciones comunes: conversión de tipos, manejo de nulos, renombrado, agregaciones y merges.
- Ejemplo completo de ETL: leer CSV -> limpiar/transformar -> exportar a SQLite.

El cuaderno está pensado para usarse en clase: ejecutar celda por celda, experimentar y comentar resultados.

In [2]:
# Imports básicos
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# Mostrar versión
pd.__version__

'2.3.3'

## Conceptos básicos

- **Series**: 1-dim (etiquetas + valores).
- **DataFrame**: 2-dim, tabla con filas y columnas.

Práctica: crear Series y DataFrame pequeño.

In [3]:
# Series y DataFrame de ejemplo
s = pd.Series([10,20,30], index=['a','b','c'])
print('Series:\n', s)

df = pd.DataFrame({'producto':['A','B','C'], 'precio':[100,150,200], 'vendidos':[5,3,8]})
print('\nDataFrame:\n', df)

df

Series:
 a    10
b    20
c    30
dtype: int64

DataFrame:
   producto  precio  vendidos
0        A     100         5
1        B     150         3
2        C     200         8


,producto,precio,vendidos
0,A,100,5
1,B,150,3
2,C,200,8


## Leer datos (I/O)

- `pd.read_csv()` para CSV.
- `df.head()`, `df.info()`, `df.describe()` y `df.dtypes` para inspección rápida.

A continuación cargamos un CSV de ejemplo (ya incluido en este cuaderno).

In [4]:
# Creamos un CSV de ejemplo para la práctica
csv_path = 'sample_sales2.csv'
sample = pd.DataFrame({
    'order_id': [1001,1002,1003,1004,1005,1006],
    'date': ['2025-09-30','2025-09-30','2025-10-01','2025-10-01','2025-10-02','2025-10-02'],
    'customer': ['Juan','María','Carlos','Ana','María','Juan'],
    'product': ['Panel A','Panel B','Inversor X','Panel A','Inversor X','Panel B'],
    'quantity': [2,1,1,3,2,1],
    'unit_price': [250.0,300.0,1200.0,250.0,1200.0,300.0],
    'region': ['Bogotá','Medellín','Bogotá','Cali','Bogotá','Medellín']
})
sample.to_csv(csv_path, index=False)
print('CSV creado en', csv_path)
sample.head()

CSV creado en sample_sales2.csv


,order_id,date,customer,product,quantity,unit_price,region
0,1001,2025-09-30,Juan,Panel A,2,250.0,Bogotá
1,1002,2025-09-30,María,Panel B,1,300.0,Medellín
2,1003,2025-10-01,Carlos,Inversor X,1,1200.0,Bogotá
3,1004,2025-10-01,Ana,Panel A,3,250.0,Cali
4,1005,2025-10-02,María,Inversor X,2,1200.0,Bogotá


In [5]:
# Leer el CSV y hacer inspección básica
df = pd.read_csv(csv_path)
print('Primeras filas:')
print(df.head())

print('\nInfo:')
df.info()

print('\nDescripción numérica:')
print(df.describe())

Primeras filas:
   order_id        date customer     product  quantity  unit_price    region
0      1001  2025-09-30     Juan     Panel A         2       250.0    Bogotá
1      1002  2025-09-30    María     Panel B         1       300.0  Medellín
2      1003  2025-10-01   Carlos  Inversor X         1      1200.0    Bogotá
3      1004  2025-10-01      Ana     Panel A         3       250.0      Cali
4      1005  2025-10-02    María  Inversor X         2      1200.0    Bogotá

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   order_id    6 non-null      int64  
 1   date        6 non-null      object 
 2   customer    6 non-null      object 
 3   product     6 non-null      object 
 4   quantity    6 non-null      int64  
 5   unit_price  6 non-null      float64
 6   region      6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory

### Tipos de datos y fechas

- Convierte columnas con `astype()`.
- Para fechas usar `pd.to_datetime()` y luego `dt` accessor para extraer día/mes/año.


In [6]:
# Convertir tipos y trabajar con fechas
print('Antes:', df.dtypes, sep='\n')

df['date'] = pd.to_datetime(df['date'])
df['quantity'] = df['quantity'].astype(int)
df['unit_price'] = df['unit_price'].astype(float)

print('\nDespués:', df.dtypes, sep='\n')

# Extraer columnas de fecha
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
print('\nCon columnas de fecha:\n', df.head())

Antes:
order_id        int64
date           object
customer       object
product        object
quantity        int64
unit_price    float64
region         object
dtype: object

Después:
order_id               int64
date          datetime64[ns]
customer              object
product               object
quantity               int64
unit_price           float64
region                object
dtype: object

Con columnas de fecha:
    order_id       date customer     product  quantity  unit_price    region  \
0      1001 2025-09-30     Juan     Panel A         2       250.0    Bogotá   
1      1002 2025-09-30    María     Panel B         1       300.0  Medellín   
2      1003 2025-10-01   Carlos  Inversor X         1      1200.0    Bogotá   
3      1004 2025-10-01      Ana     Panel A         3       250.0      Cali   
4      1005 2025-10-02    María  Inversor X         2      1200.0    Bogotá   

   year  month  
0  2025      9  
1  2025      9  
2  2025     10  
3  2025     10  
4  2025     1

### Valores faltantes y duplicados

- `df.isna().sum()` para contar nulos.
- `df.dropna()` o `df.fillna()` para tratarlos.
- `df.duplicated()` y `df.drop_duplicates()` para duplicados.


In [7]:
# Simular nulos y duplicados para demostración
# Agregamos una fila con nulos y una duplicada
extra = pd.DataFrame([{
    'order_id':1007, 'date':pd.NaT, 'customer':None, 'product':'Panel A', 'quantity':np.nan, 'unit_price':250.0, 'region':'Cali'
}])
df2 = pd.concat([df, extra, df.iloc[[0]]], ignore_index=True)
print(df2)

print('\nNulos por columna:\n', df2.isna().sum())

# Ejemplos de tratamiento
cleaned = df2.dropna(subset=['date','customer'])
filled = df2.fillna({'quantity':0, 'customer':'Desconocido'})
print('\nDropna result:\n', cleaned)
print('\nFillna result:\n', filled)

# Duplicados
print('\nDuplicados encontrados:', df2.duplicated().sum())
no_dup = df2.drop_duplicates()
print('\nSin duplicados:\n', no_dup)

   order_id       date customer     product  quantity  unit_price    region  \
0      1001 2025-09-30     Juan     Panel A       2.0       250.0    Bogotá   
1      1002 2025-09-30    María     Panel B       1.0       300.0  Medellín   
2      1003 2025-10-01   Carlos  Inversor X       1.0      1200.0    Bogotá   
3      1004 2025-10-01      Ana     Panel A       3.0       250.0      Cali   
4      1005 2025-10-02    María  Inversor X       2.0      1200.0    Bogotá   
5      1006 2025-10-02     Juan     Panel B       1.0       300.0  Medellín   
6      1007        NaT     None     Panel A       NaN       250.0      Cali   
7      1001 2025-09-30     Juan     Panel A       2.0       250.0    Bogotá   

     year  month  
0  2025.0    9.0  
1  2025.0    9.0  
2  2025.0   10.0  
3  2025.0   10.0  
4  2025.0   10.0  
5  2025.0   10.0  
6     NaN    NaN  
7  2025.0    9.0  

Nulos por columna:
 order_id      0
date          1
customer      1
product       0
quantity      1
unit_price    0


### Selección, filtrado y renombrado

- `df[['col1','col2']]` para seleccionar columnas.
- `df.loc[condición]` y `df.query()` para filtrar.
- `df.rename(columns={...})` para renombrar.


In [8]:
# Selección y filtrado
# Seleccionar columnas
cols = df[['order_id','date','customer','product','quantity','unit_price']]
print(cols.head())

# Filtrar: ventas en Bogotá con quantity >= 2
bogota_mayor2 = df[(df['region']=='Bogotá') & (df['quantity']>=2)]
print('\nFiltrado:\n', bogota_mayor2)

# Renombrar columnas
df_renamed = df.rename(columns={'unit_price':'price_unit'})
print('\nRenombrado columnas:\n', df_renamed.head())

   order_id       date customer     product  quantity  unit_price
0      1001 2025-09-30     Juan     Panel A         2       250.0
1      1002 2025-09-30    María     Panel B         1       300.0
2      1003 2025-10-01   Carlos  Inversor X         1      1200.0
3      1004 2025-10-01      Ana     Panel A         3       250.0
4      1005 2025-10-02    María  Inversor X         2      1200.0

Filtrado:
    order_id       date customer     product  quantity  unit_price  region  \
0      1001 2025-09-30     Juan     Panel A         2       250.0  Bogotá   
4      1005 2025-10-02    María  Inversor X         2      1200.0  Bogotá   

   year  month  
0  2025      9  
4  2025     10  

Renombrado columnas:
    order_id       date customer     product  quantity  price_unit    region  \
0      1001 2025-09-30     Juan     Panel A         2       250.0    Bogotá   
1      1002 2025-09-30    María     Panel B         1       300.0  Medellín   
2      1003 2025-10-01   Carlos  Inversor X      

### Nuevas columnas y operaciones vectorizadas

- Evitar ciclos. Usar operaciones sobre columnas enteras (vectorizadas).
- Ejemplo: `df['total'] = df['quantity'] * df['unit_price']`.


In [10]:
# Crear columna 'total' y otras transformaciones
_df = df.copy()
_df['total'] = _df['quantity'] * _df['unit_price']
_df['customer_lower'] = _df['customer'].str.lower()
print(_df)

# Operaciones rápidas: ordenar
print('\nOrdenado por total desc:\n', _df.sort_values('total', ascending=False))

   order_id       date customer     product  quantity  unit_price    region  \
0      1001 2025-09-30     Juan     Panel A         2       250.0    Bogotá   
1      1002 2025-09-30    María     Panel B         1       300.0  Medellín   
2      1003 2025-10-01   Carlos  Inversor X         1      1200.0    Bogotá   
3      1004 2025-10-01      Ana     Panel A         3       250.0      Cali   
4      1005 2025-10-02    María  Inversor X         2      1200.0    Bogotá   
5      1006 2025-10-02     Juan     Panel B         1       300.0  Medellín   

   year  month   total customer_lower  
0  2025      9   500.0           juan  
1  2025      9   300.0          maría  
2  2025     10  1200.0         carlos  
3  2025     10   750.0            ana  
4  2025     10  2400.0          maría  
5  2025     10   300.0           juan  

Ordenado por total desc:
    order_id       date customer     product  quantity  unit_price    region  \
4      1005 2025-10-02    María  Inversor X         2      1

### Agrupaciones y agregaciones (`groupby`)

- `groupby` + `agg` para sumar, contar, promedio, etc.
- `pivot_table` para tablas pivote.


In [11]:
# Agrupar por producto y región
gb = _df.groupby(['product','region']).agg(
    total_qty = ('quantity','sum'),
    sales_value = ('total','sum'),
    orders = ('order_id','count')
).reset_index()
print(gb)

# Pivot table: ventas por producto x región
pivot = pd.pivot_table(_df, index='product', columns='region', values='total', aggfunc='sum', fill_value=0)
print('\nPivot table:\n', pivot)

      product    region  total_qty  sales_value  orders
0  Inversor X    Bogotá          3       3600.0       2
1     Panel A    Bogotá          2        500.0       1
2     Panel A      Cali          3        750.0       1
3     Panel B  Medellín          2        600.0       2

Pivot table:
 region      Bogotá   Cali  Medellín
product                            
Inversor X  3600.0    0.0       0.0
Panel A      500.0  750.0       0.0
Panel B        0.0    0.0     600.0


### Merge / Join

- Unir DataFrames con `pd.merge()` (inner, left, right, outer).
- Útil cuando traes catálogos o dimensiones desde otros archivos.


In [12]:
# Ejemplo de merge: catálogo de productos
products = pd.DataFrame({'product':['Panel A','Panel B','Inversor X'], 'category':['Panel','Panel','Inversor'], 'weight_kg':[15,17,8]})
print('Products catalog:\n', products)
merged = pd.merge(_df, products, on='product', how='left')
print('\nMerged:\n', merged)

Products catalog:
       product  category  weight_kg
0     Panel A     Panel         15
1     Panel B     Panel         17
2  Inversor X  Inversor          8

Merged:
    order_id       date customer     product  quantity  unit_price    region  \
0      1001 2025-09-30     Juan     Panel A         2       250.0    Bogotá   
1      1002 2025-09-30    María     Panel B         1       300.0  Medellín   
2      1003 2025-10-01   Carlos  Inversor X         1      1200.0    Bogotá   
3      1004 2025-10-01      Ana     Panel A         3       250.0      Cali   
4      1005 2025-10-02    María  Inversor X         2      1200.0    Bogotá   
5      1006 2025-10-02     Juan     Panel B         1       300.0  Medellín   

   year  month   total customer_lower  category  weight_kg  
0  2025      9   500.0           juan     Panel         15  
1  2025      9   300.0          maría     Panel         17  
2  2025     10  1200.0         carlos  Inversor          8  
3  2025     10   750.0           

### `apply`, `map`, `transform`

- `apply` para aplicar funciones fila a fila o columna a columna.
- Prefiere funciones vectorizadas; `apply` suele ser más lento, pero útil cuando no hay alternativa.


In [13]:
# Uso de apply para crear una columna categórica
merged['category_short'] = merged['category'].apply(lambda x: 'Inv' if x=='Inversor' else 'Pan')
print(merged[['product','category','category_short']])

      product  category category_short
0     Panel A     Panel            Pan
1     Panel B     Panel            Pan
2  Inversor X  Inversor            Inv
3     Panel A     Panel            Pan
4  Inversor X  Inversor            Inv
5     Panel B     Panel            Pan


## ETL: ejemplo completo

1. **Extract**: leer CSV.
2. **Transform**: limpiar nulos, corregir tipos, crear columnas, agregar/filtrar.
3. **Load**: exportar a base de datos (SQLite) usando `to_sql`.

A continuación haremos todo el flujo y dejaremos una tabla `sales_clean` lista para subir a cualquier SGBD.

In [ ]:
# ETL completo
# Extract
raw = pd.read_csv(csv_path)

# Transform
raw['date'] = pd.to_datetime(raw['date'], errors='coerce')
raw = raw.dropna(subset=['date'])  # quitar filas sin fecha
raw['quantity'] = raw['quantity'].fillna(0).astype(int)
raw['unit_price'] = raw['unit_price'].astype(float)
raw['total'] = raw['quantity'] * raw['unit_price']

# Normalizar nombres de cliente
raw['customer'] = raw['customer'].str.strip().fillna('Desconocido')

# Agregar identificador único si no existe
if 'id' not in raw.columns:
    raw = raw.reset_index().rename(columns={'index':'id'})

# Seleccionar columnas finales
final = raw[['id','order_id','date','customer','product','quantity','unit_price','total','region']]
print('Final ETL result:')
print(final)

# Load -> SQLite (ejemplo local)
engine = create_engine('sqlite:////mnt/data/sales_etl.db')
final.to_sql('sales_clean', con=engine, if_exists='replace', index=False)
print('\nTabla `sales_clean` escrita en SQLite en /mnt/data/sales_etl.db')

In [ ]:
# Leer desde SQLite para confirmar
engine = create_engine('sqlite:////mnt/data/sales_etl.db')
check = pd.read_sql('sales_clean', con=engine)
print(check.head())

### Exportar y siguientes pasos

- `df.to_csv()` para CSV.
- `df.to_parquet()` para formatos colunar (más eficientes).
- `df.to_sql()` para cargar directamente a bases de datos.

Consejo: validar esquemas y tipos antes de subir a producción; usar migraciones o scripts SQL para crear tablas con tipos fijos.

## Sugerencias de ejercicios para la clase

1. Añadir una columna `discount` que dependa del valor total y aplicarla.
2. Detectar clientes recurrentes y crear una tabla `customers` con recuento de órdenes.
3. Simular datos faltantes y practicar `fillna` y `interpolate`.
4. Hacer ETL para varias fuentes (CSV + Excel) y unirlas.

---

Fin del cuaderno. Si quieres, puedo:
- Ajustar el cuaderno para tus datos reales.
- Añadir una sección sobre `dtypes` y validación con `pandera`.
- Generar una presentación con los puntos principales para tus diapositivas.